In [1]:
# Import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, make_scorer, classification_report
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
import mord
from collections import Counter

Using TensorFlow backend.


In [2]:
path = "/Users/Stijn/Documents/Master Data Science and Society/Block 3/thesis/code/thesis_Code/"
mood = pd.read_csv(path+'mood_imputed_median.csv', sep = ',', index_col=0)
mood['content_last'] = mood.groupby('user_id')['content'].shift()
mood.loc[(pd.isnull(mood.content_last)), 'content_last'] = mood['content']
mood_content = mood.drop(["anxious", "bored", "cheerful", "user_id", "response_time"], axis=1)

In [3]:
mood_content[:5]

,content,day_time_window,average_TimeUse,bulk,messaging,socialnetworking,otherapp,content_last
4825,4.0,3.0,0.673000,0,0,0,1,4.0
4826,4.0,4.0,64.314667,0,1,0,2,4.0
245,4.0,1.0,20.590500,0,0,0,2,4.0
4827,4.0,2.0,0.000000,0,0,0,0,4.0
4828,4.0,3.0,53.957200,0,3,0,2,4.0


In [4]:
# Normalize predictors
mood_normz = mood_content.copy()
features_to_normalize = ['day_time_window', 'average_TimeUse', 'messaging', 'socialnetworking', 'otherapp']
mood_normz[features_to_normalize] = mood_normz[features_to_normalize].apply(lambda x:(x-x.min()) / (x.max()-x.min()))

In [5]:
# X = predictors, y = target
X = mood_normz.iloc[:,1:]
y = mood_normz.iloc[:, 0:1]

# Convert float to int
y = y.astype(int)

In [6]:
# Train (80%) & Test set (20%)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [7]:
# Oversampling unbalanced target
print("Before OverSampling, counts of labels: {}".format(y['content'].value_counts()))
  
sm = SMOTENC(categorical_features=[0, 2, 6], random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(X, y['content'].ravel()) 

print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0))) 
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1))) 
print("After OverSampling, counts of label '2': {}".format(sum(y_train_res == 2))) 
print("After OverSampling, counts of label '3': {}".format(sum(y_train_res == 3))) 
print("After OverSampling, counts of label '4': {}".format(sum(y_train_res == 4))) 
print("After OverSampling, counts of label '5': {}".format(sum(y_train_res == 5))) 

Before OverSampling, counts of labels: 3    3972
4    2958
2    2775
1    1559
0    1090
5     867
Name: content, dtype: int64
After OverSampling, counts of label '0': 3972
After OverSampling, counts of label '1': 3972
After OverSampling, counts of label '2': 3972
After OverSampling, counts of label '3': 3972
After OverSampling, counts of label '4': 3972
After OverSampling, counts of label '5': 3972


In [8]:
# Convert oversample dataset to Pandas DataFrame
X_train = pd.DataFrame(data=X_train_res)
y_train = pd.DataFrame(data=y_train_res)

In [9]:
X_train_nomood = X_train.iloc[:,:-1]
X_train_onlyboredlast = X_train.iloc[:,-1:]
# X_test_nomood = X_test.iloc[:,:-1]
# X_test_onlyboredlast = X_test.iloc[:,-1:]

In [10]:
# X_test_bored = X_test_boredseries.values

X_train = X_train_nomood.values
y_train = y_train.values.ravel()

In [11]:
len(y_train)

23832

In [12]:
# Shuffle both train and test set to cancel patterns
X, y = shuffle(X_train, y_train)
#X_te, y_te = shuffle(X_test_nomood, y_test)

In [13]:
parameter_candidates = {'max_iter': [10, 100, 250, 500], 'alpha': [0.01, 0.1, 0.5, 1.0]}
clf = GridSearchCV(estimator=mord.LogisticAT(), 
                    param_grid=parameter_candidates,
                    cv=10,
                   scoring = 'neg_mean_squared_error',
                    n_jobs=-1)

In [14]:
clf_fit = clf.fit(X, y)

In [15]:
print('Best score:', clf_fit.best_score_) 
print(" ")
print('Best estimator:',clf_fit.best_estimator_) 

Best score: -3.0468277945619335
 
Best estimator: LogisticAT(alpha=0.01, max_iter=250, verbose=0)


In [16]:
def macroaverage_mae(test, pred):
    mae_0, mae_1, mae_2, mae_3, mae_4, mae_5 = (0,0,0,0,0,0)

    for i in range(len(pred)):
        if test[i] == 0 and pred[i] != 0:
            mae_0 += (abs(test[i]-pred[i]))
        if test[i] == 1 and pred[i] != 1:
            mae_1 += (abs(test[i]-pred[i]))
        if test[i] == 2 and pred[i] != 2:
            mae_2 += (abs(test[i]-pred[i]))
        if test[i] == 3 and pred[i] != 3:
            mae_3 += (abs(test[i]-pred[i]))
        if test[i] == 4 and pred[i] != 4:
            mae_4 += (abs(test[i]-pred[i]))
        if test[i] == 5 and pred[i] != 5:
            mae_5 += (abs(test[i]-pred[i]))

    cnt_0, cnt_1, cnt_2, cnt_3, cnt_4, cnt_5 = (0,0,0,0,0,0)
    for z in range(len(test)):
        if test[z] == 0:
            cnt_0 += 1
        if test[z] == 1:
            cnt_1 += 1
        if test[z] == 2:
            cnt_2 += 1
        if test[z] == 3:
            cnt_3 += 1
        if test[z] == 4:
            cnt_4 += 1
        if test[z] == 5:
            cnt_5 += 1

    mae_macroaverage = ((mae_0/cnt_0) + (mae_1/cnt_1) + (mae_2/cnt_2) + (mae_3/cnt_3) + (mae_4/cnt_4) + (mae_5/cnt_5)) / 6
    return mae_macroaverage 

#score = make_scorer(macroaverage_mae, greater_is_better=False)

In [17]:
# X = predictors, y = target
X = mood_normz.iloc[:,1:-1]
y = mood_normz.iloc[:, 0:1]

# Convert float to int
y = y.astype(int)

In [18]:
X_train, y_train = shuffle(X, y)

In [19]:
# Convert oversample dataset to Pandas DataFrame
X = pd.DataFrame(data=X_train)
y = pd.DataFrame(data=y_train)

In [20]:
n_split=10
f1_scores = []
f1_scores_classes = []
mmae_scores = []
for train_index, test_index in KFold(n_splits = n_split, random_state=2, shuffle=True).split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    sm = SMOTENC(categorical_features=[0, 2], random_state = 2) 
    X_train_oversampl, y_train_oversampl = sm.fit_sample(X_train, y_train['content'].ravel()) 
    X_use, y_use = shuffle(X_train_oversampl, y_train_oversampl)
    
    model = mord.LogisticAT(alpha=0.01, max_iter=250, verbose=0)
    model.fit(X_use, y_use)   
    pred = model.predict(X_test)
    print("Predicted values: ", Counter(pred))
    
    f1_scores.append(round(f1_score(y_test, pred, average='weighted'), 4))
    f1_scores_classes.append(f1_score(y_test, pred, average=None))
    
    test = y_test.values
    mmae_scores.append(macroaverage_mae(test, pred))

Predicted values:  Counter({2: 825, 3: 483, 4: 13, 1: 2})


/Users/Stijn/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Predicted values:  Counter({2: 803, 3: 503, 4: 14, 5: 1, 1: 1})


/Users/Stijn/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Predicted values:  Counter({2: 859, 3: 451, 4: 12})


...

/Users/Stijn/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Predicted values:  Counter({3: 670, 2: 639, 4: 11, 1: 2})


/Users/Stijn/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Predicted values:  Counter({2: 797, 3: 514, 4: 10, 5: 1})


/Users/Stijn/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
print("Mean F1-score per class: ", [round(np.mean(x), 4) for x in zip(*f1_scores_classes)])
print("STD F1-score per class: ", [round(np.std(x), 4) for x in zip(*f1_scores_classes)])

Mean F1-score per class:  [0.0, 0.0012, 0.3183, 0.3695, 0.0195, 0.0]
STD F1-score per class:  [0.0, 0.0036, 0.0213, 0.0237, 0.0093, 0.0]


In [22]:
print("Macro-averaged F1-scores: ", f1_scores)
print("Mean MA F1-scores: ", round(np.mean(f1_scores), 4))
print("Std MA F1-scores: ", round(np.std(f1_scores), 4))

Macro-averaged F1-scores:  [0.1815, 0.1914, 0.1733, 0.1751, 0.1801, 0.1795, 0.1979, 0.1643, 0.1919, 0.189]
Mean MA F1-scores:  0.1824
Std MA F1-scores:  0.0097


In [23]:
print("Macro-averaged MAE: ", mmae_scores)
print("Mean MMAE-scores: ", round(np.mean(mmae_scores), 4))
print("Std MMAE-scores: ", round(np.std(mmae_scores), 4))

Macro-averaged MAE:  [array([1.45455657]), array([1.44816838]), array([1.48362849]), array([1.48608228]), array([1.46348997]), array([1.50209458]), array([1.47521039]), array([1.46912139]), array([1.49201546]), array([1.44450794])]
Mean MMAE-scores:  1.4719
Std MMAE-scores:  0.0183
